In [1]:
#!pip install azure
#!pip install azure-ai-formrecognizer


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\devik\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\devik\\AppData\\Local\\Temp\\pip-install-t5sbf8ig\\azure_2ee21eb0e9a34c72b2d0ab7e094fe7c2\\setup.py'"'"'; __file__='"'"'C:\\Users\\devik\\AppData\\Local\\Temp\\pip-install-t5sbf8ig\\azure_2ee21eb0e9a34c72b2d0ab7e094fe7c2\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\devik\AppData\Local\Temp\pip-pip-egg-info-yl4af13v'
         cwd: C:\Users\devik\AppData\Local\Temp\pip-install-t5sbf8ig\azure_2ee21eb0e9a34c72b2d0ab7e094fe7c2\
    Complete output (24 lines):
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "C:\Users\devik\AppData\Local\Temp\pip-install-t5sbf8ig\azure_2ee21eb0e9a34c72b2d0ab7e094fe7c2\setup.py", line 60, i

ERROR: unknown command " install openai" - maybe you meant "install"



In [4]:
#!pip install openai

In [1]:
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from openai import AzureOpenAI

# Initialize Azure Form Recognizer client
form_recognizer_endpoint = "https://spjdocumentintelligence.cognitiveservices.azure.com/"
form_recognizer_key = "88c92c39f1794aafb2c84d2d7826f9d6"
form_recognizer_client = DocumentAnalysisClient(endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key))

# Initialize OpenAI client
openai_api_key = "7d8a1f9f0ace4c19ac8fb19c82827521"
openai_endpoint = "https://spjopenai9902916927.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
openai_client = AzureOpenAI(api_key=openai_api_key, api_version="2023-07-01-preview", azure_endpoint=openai_endpoint)

def extract_dataframe_from_pdf(file_path):
    # Read file content
    with open(file_path, "rb") as file:
        file_content = file.read()

    # Analyze document
    poller = form_recognizer_client.begin_analyze_document("prebuilt-document", file_content)
    result = poller.result()

    # Extract table data
    table_data = []
    for table in result.tables:
        table_dict = {}
        for cell in table.cells:
            if cell.row_index not in table_dict:
                table_dict[cell.row_index] = {}
            table_dict[cell.row_index][cell.column_index] = cell.content.strip()
        table_data.append(table_dict)

    # Convert table data to DataFrame
    dfs = []
    for table in table_data:
        headers = sorted(table[0].keys())  # Use sorted headers as column names
        df = pd.DataFrame.from_dict(table, orient='index', columns=headers)
        dfs.append(df)

    # Return the first DataFrame (assuming there's only one table in the PDF)
    if dfs:
        return df.rename(columns=df.iloc[0]).drop(df.index[0])
    else:
        return None

def extract_summary_from_dataframe(openai_client, df):
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    prompt = "You are a medical blood test report analyzer. Please extract the test values that are higher or lower than the reference range. Give the ouput in a way which is friendly.Dont mention the word friendly"
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    response = openai_client.chat.completions.create(
        model="gpt-4-medical",
        temperature=0.3,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=800,
        messages=[
            {"role": "system", "content": "Assistant is a medical blood test report analyzer."},
            {"role": "user", "content": f"{prompt}"},
            {"role": "assistant", "content": f"{text}"}
        ]
    )
    return response.choices[0].message.content.strip()



# Example usage
file_path = r"C:\Users\devik\Desktop\Assignments\capstone\blood report samples.pdf"
df = extract_dataframe_from_pdf(file_path)
if df is not None:
    print("HEY THERE! THESE ARE YOUR TEST RESULTS:")
    print(df)
    summary = extract_summary_from_dataframe(openai_client, df)
    print(summary)
else:
    print("No DataFrame extracted from PDF.")
    


HEY THERE! THESE ARE YOUR TEST RESULTS:
     Test Name Result Normal Range    Units
1   Hemoglobin     12  11.0 - 16.0     g/dL
2          RBC    3.3     3,5-5.50  10~6/UL
3          HCT     36    37.0-50.0        %
4          MCV     83        82-95       fl
5          MCH     28        27-31       pg
6         MCHC     33    32.0-36.0     g/dL
7       RDW-CV     12    11.5-14.5        %
8       RDW-SD     44        35-56       fl
9          WBC    6.7       4.5-11  10~3/uL
10        NEU%     60        40-70        %
11        LYM%     30        20-45        %
12        MON%      8         2-10        %
13        EOS%      2          1-6        %
14        BAS%      0          0-2        %
15        LYM#      2      1.5-4.0  10^3/uL
16        GRA#    4.7      2.0-7.5  10^3/uL
17         PLT    256      150-450  10^3/uL
18         ESR    N 2     Up to 15    mm/hr
Sure, let's go through your blood test results:

1. Red Blood Cell (RBC) count is slightly low at 3.3 (normal range is 3.5-5

Phase 2 

In [8]:
import pandas as pd
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from openai import AzureOpenAI

# Initialize Azure Form Recognizer client
form_recognizer_endpoint = "https://spjdocumentintelligence.cognitiveservices.azure.com/"
form_recognizer_key = "88c92c39f1794aafb2c84d2d7826f9d6"
form_recognizer_client = DocumentAnalysisClient(endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key))

# Initialize OpenAI client
openai_api_key = "7d8a1f9f0ace4c19ac8fb19c82827521"
openai_endpoint = "https://spjopenai9902916927.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
openai_client = AzureOpenAI(api_key=openai_api_key, api_version="2023-07-01-preview", azure_endpoint=openai_endpoint)

def extract_dataframe_from_pdf(file_path):
    # Read file content
    with open(file_path, "rb") as file:
        file_content = file.read()

    # Analyze document
    poller = form_recognizer_client.begin_analyze_document("prebuilt-document", file_content)
    result = poller.result()

    # Extract table data
    table_data = []
    for table in result.tables:
        table_dict = {}
        for cell in table.cells:
            if cell.row_index not in table_dict:
                table_dict[cell.row_index] = {}
            table_dict[cell.row_index][cell.column_index] = cell.content.strip()
        table_data.append(table_dict)

    # Convert table data to DataFrame
    dfs = []
    for table in table_data:
        headers = sorted(table[0].keys())  # Use sorted headers as column names
        df = pd.DataFrame.from_dict(table, orient='index', columns=headers)
        dfs.append(df)

    # Return the first DataFrame (assuming there's only one table in the PDF)
    if dfs:
        return df.rename(columns=df.iloc[0]).drop(df.index[0])
    else:
        return None

def suggest_nutrient_intake(openai_client, df):
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    prompt = "You are a medical blood test report analyzer and nutrition expert. See for the test values if they are higher or lower than the reference range. But dont give give this as the ouput. your output should be the nutrients that we should intake for the observations from the analysis. You should only provide the information from the national institute of health us information website.Can you make the format in a better way. easily readable and has some designs. Only provide the nurtition information for those results which are either high or low."
    # Convert DataFrame to text
    text = df.to_string(index=False, header=True)
    response = openai_client.chat.completions.create(
        model="gpt-4-medical",
        temperature=0.3,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=800,
        messages=[
            {"role": "system", "content": "Assistant is a medical blood test report analyzer and nutrition expert."},
            {"role": "user", "content": f"{prompt}"},
            {"role": "assistant", "content": f"{text}"}
        ]
    )
    return response.choices[0].message.content.strip()

# Example usage
file_path = r"C:\Users\devik\Desktop\Assignments\capstone\blood report samples.pdf"
df = extract_dataframe_from_pdf(file_path)
if df is not None:

    nutrient = suggest_nutrient_intake(openai_client, df)
    print(nutrient)
else:
    print("No DataFrame extracted from PDF.")
    


Based on your blood test results, here are the nutrients you should focus on according to the National Institute of Health:

1. **Iron, Vitamin B12, and Folic Acid**: Your Hemoglobin and RBC (Red Blood Cells) are slightly lower than the normal range. These nutrients are essential for the production of red blood cells and hemoglobin.

2. **Vitamin C and Iron**: Your HCT (Hematocrit) is slightly lower than the normal range. These nutrients can help in increasing the percentage of red blood cells in your blood.

3. **Vitamin B12 and Folic Acid**: Your LYM# (Absolute Lymphocyte Count) is slightly lower than the normal range. These nutrients are essential for the production and function of white blood cells.

Remember, it's important to maintain a balanced diet and consult with a healthcare provider before starting any new supplement regimen.


In [ ]:
### extra file 



from openai import AzureOpenAI

# Initialize OpenAI client
openai_api_key = "7d8a1f9f0ace4c19ac8fb19c82827521"
openai_endpoint = "https://spjopenai9902916927.openai.azure.com/openai/deployments/gpt-4-32k/chat/completions?api-version=2023-07-01-preview"
openai_client = AzureOpenAI(api_key=openai_api_key, api_version="2023-07-01-preview", azure_endpoint=openai_endpoint)


def suggest_nutrient_intake(summary):
    prompt = "You are a nutrient intake suggestion assistant. U need to suggest nutrients that should be intaken based on the summary you extracted earlier"
    response = openai_client.chat.completions.create(
        model="gpt-4",
        temperature=0.3,
        top_p=0.95,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        max_tokens=400,
        messages=[
            {"role": "system", "content": "Assistant providing nutrient intake suggestions based on summary."},
            {"role": "user", "content": f"{prompt}"}
        ]
    )

    return response.choices[0].message.content.strip()

# Example usage
summary = extract_summary_from_dataframe(openai_client, df)
nutrient_intake_suggestions = suggest_nutrient_intake(summary)
print("Nutrient Intake Suggestions:", nutrient_intake_suggestions)
